ABC-анализ — это метод управления и классификации запасов, задач или клиентов по степени важности, основанный на принципе Парето (правило 80/20). В этом анализе элементы делятся на три категории:

* Категория A — самые важные элементы, которые составляют примерно 20% от общего количества, но приносят около 80% ценности (например, доходов или затрат).
* Категория B — элементы средней важности, составляющие около 30% от общего числа и обеспечивающие примерно 15% ценности.
* Категория C — наименее важные элементы, которые составляют 50% общего числа, но приносят лишь около 5% ценности.

Проведем многомерный ABC анализ по:
* количеству проданных позиций,
* прибыли с позиции,
* выручке.

Для проведения анализа воспользуемся таблицей **apteka.sales**. содержащей данные о продажах аптечной сети. Интересующие нас поля:
* dr_kol - количество проданного товара в данной строке чека,
* dr_ndrugs - наименование товара,
* dr_croz - розничная цена,
* dr_czak - закупочная цена,
* dr_sdisc - сумма скидки на всю строку чека.

In [1]:
import pandas as pd
from sqlalchemy import text
from sqlalchemy import create_engine
from google.colab import userdata
SQLALCHEMY_SILENCE_UBER_WARNING=1

In [2]:
engine = create_engine(userdata.get('supabase'))

In [3]:
def select(sql):
    sql = text(sql)
    return pd.read_sql(sql, engine)

In [4]:
sql = """
-- Сгруппируем товары по количесвтву продаж
with product_grouped as (
select
  dr_ndrugs, sum(dr_kol) as amounts,
  sum((dr_croz - dr_czak) * dr_kol - dr_sdisc) as pdofit_sum,
  sum(dr_croz * dr_kol - dr_sdisc) as revenue_sum
from
  apteka.sales
group by dr_ndrugs
)


-- Выполним ABC анализ по количеству (A-B-C  => 80-15-5%)
select product_grouped.dr_ndrugs,
case
    when sum(product_grouped.amounts) over (order by amounts desc) <= (select sum(dr_kol) * 0.8 from apteka.sales) then 'A'
    when sum(product_grouped.amounts) over (order by amounts desc) <= (select sum(dr_kol) * 0.95 from apteka.sales) then 'B'
    else 'C'
end as group_amount,
case
    when sum(product_grouped.pdofit_sum) over (order by pdofit_sum desc) <= (select sum((dr_croz - dr_czak) * dr_kol - dr_sdisc)* 0.8 from apteka.sales) then 'A'
    when sum(product_grouped.pdofit_sum) over (order by pdofit_sum desc) <= (select sum((dr_croz - dr_czak) * dr_kol - dr_sdisc)* 0.95 from apteka.sales) then 'B'
    else 'C'
end as group_profit,
case
    when sum(product_grouped.revenue_sum) over (order by revenue_sum desc) <= (select sum(dr_croz * dr_kol - dr_sdisc)* 0.8 from apteka.sales) then 'A'
    when sum(product_grouped.revenue_sum) over (order by revenue_sum desc) <= (select sum(dr_croz * dr_kol - dr_sdisc)* 0.95 from apteka.sales) then 'B'
    else 'C'
end as group_revenue

from product_grouped
limit 1000
"""

In [5]:
select(sql)

,dr_ndrugs,group_amount,group_profit,group_revenue
0,ПАКЕТ,A,A,A
1,"ЛЕЙКОПЛАСТЫРЬ БАКТЕР. 2,5Х7,2 №1 /ВЕРОФАРМ/",A,A,A
2,"НАФТИЗИН 0,1% 15МЛ. НАЗАЛ.КАПЛИ ФЛ./КАП. /СЛАВ...",A,A,A
3,КОРВАЛОЛ 25МЛ. КАПЛИ Д/ПРИЕМА ВНУТРЬ ФЛ. И/У /...,A,A,A
4,"СНУП 0,1% 90МКГ/ДОЗА 15МЛ. НАЗАЛ.СПРЕЙ ФЛ. /ШТ...",A,A,A
...,...,...,...,...
995,ГИПОСАРТ 8МГ. №28 ТАБ.,A,A,A
996,САЛФЕТКИ 8-И СЛОЙНАЯ СТЕР. 5СМХ5СМ №10,A,B,C
997,РОТОКАН 50МЛ. ЭКСТРАКТ Д/ПРИЕМА ВНУТРЬ И МЕСТ....,A,B,B
998,АСКОРБИНОВАЯ К-ТА 25МГ. ЛИМОН №10 ТАБ. КРУТКА ...,A,C,C
